<a href="https://colab.research.google.com/github/41371109h/114-1/blob/main/hw5_%E5%8D%88%E9%A4%90%E8%81%9A%E6%9C%83%E6%B1%BA%E7%AD%96%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-search-results folium geopy -q

  Preparing metadata (setup.py) ... done


In [2]:
from serpapi import GoogleSearch
import folium
from geopy.geocoders import Nominatim
import json
import re
from datetime import datetime
from IPython.display import display

In [3]:
SERPAPI_KEY = "fa1097a76bf103c7a52fb91c2d7c58129281c4ec8b63890e88cf355d30f21668"

## 4. 定義搜尋與地理位置函數

In [4]:
# 初始化地理編碼器
geolocator = Nominatim(user_agent="search_map_app")


TAIWAN_LOCATIONS = [
    # 直轄市
    '台北', '台北市', '臺北', '臺北市', 'Taipei',
    '新北', '新北市', 'New Taipei',
    '桃園', '桃園市', 'Taoyuan',
    '台中', '台中市', '臺中', '臺中市', 'Taichung',
    '台南', '台南市', '臺南', '臺南市', 'Tainan',
    '高雄', '高雄市', 'Kaohsiung',

    # 縣市
    '基隆', '基隆市', 'Keelung',
    '新竹', '新竹市', '新竹縣', 'Hsinchu',
    '苗栗', '苗栗縣', 'Miaoli',
    '彰化', '彰化縣', 'Changhua',
    '南投', '南投縣', 'Nantou',
    '雲林', '雲林縣', 'Yunlin',
    '嘉義', '嘉義市', '嘉義縣', 'Chiayi',
    '屏東', '屏東縣', 'Pingtung',
    '宜蘭', '宜蘭縣', 'Yilan',
    '花蓮', '花蓮縣', 'Hualien',
    '台東', '台東縣', '臺東', '臺東縣', 'Taitung',
    '澎湖', '澎湖縣', 'Penghu',
    '金門', '金門縣', 'Kinmen',
    '連江', '連江縣', '馬祖', 'Matsu',

    # 知名景點/地標
    '101', '台北101', '西門町', '信義區', '中正區',
    '士林', '北投', '淡水', '九份', '平溪',
    '日月潭', '阿里山', '墾丁', '太魯閣',
    '東大門', '逢甲', '一中街', '勤美',
]

def search_google(query, api_key, num=5, gl='tw', hl='zh-tw'):
    """
    使用 SerpAPI 進行 Google 搜尋
    """
    try:
        params = {
            'q': query,
            'api_key': api_key,
            'num': num,
            'gl': gl,
            'hl': hl
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        return results.get('organic_results', [])

    except Exception as e:
        print(f"搜尋時發生錯誤: {e}")
        return []


def extract_locations_from_text(text):
    """
    從文字中擷取地區名稱

    參數:
        text (str): 要分析的文字

    回傳:
        list: 找到的地區名稱列表
    """
    found_locations = []
    text_lower = text.lower()

    for location in TAIWAN_LOCATIONS:
        if location.lower() in text_lower:
            # 避免重複加入相似地名
            base_name = location.replace('市', '').replace('縣', '')
            if base_name not in [loc.replace('市', '').replace('縣', '') for loc in found_locations]:
                found_locations.append(location)

    return found_locations


def get_coordinates(location_name):
    """
    取得地點的經緯度座標

    參數:
        location_name (str): 地點名稱

    回傳:
        tuple: (緯度, 經度) 或 None
    """
    try:
        # 加上台灣以提高準確度
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except Exception as e:
        print(f"無法取得 {location_name} 的座標: {e}")
        return None


def analyze_results_with_locations(results):
    """
    分析搜尋結果並擷取地理位置資訊

    參數:
        results (list): 搜尋結果列表

    回傳:
        list: 包含位置資訊的結果字典列表
    """
    analyzed_results = []

    for result in results:
        # 從標題和描述中尋找地點
        title = result.get('title', '')
        snippet = result.get('snippet', '')
        combined_text = f"{title} {snippet}"

        locations = extract_locations_from_text(combined_text)

        if locations:
            analyzed_results.append({
                'title': title,
                'snippet': snippet,
                'link': result.get('link', ''),
                'locations': locations
            })

    return analyzed_results


def create_map_from_results(analyzed_results, center=[23.5, 121.0], zoom=7):
    """
    根據分析結果建立地圖

    參數:
        analyzed_results (list): 包含位置資訊的結果列表
        center (list): 地圖中心座標 [緯度, 經度]
        zoom (int): 地圖縮放層級

    回傳:
        folium.Map: 地圖物件
    """
    # 建立地圖
    m = folium.Map(location=center, zoom_start=zoom, tiles='OpenStreetMap')

    # 用於記錄已標註的位置，避免重複
    marked_locations = {}

    # 為每個結果加上標記
    for result in analyzed_results:
        for location in result['locations']:
            coords = get_coordinates(location)

            if coords:
                lat, lon = coords

                # 如果該位置還沒標記過
                if location not in marked_locations:
                    marked_locations[location] = []

                # 將結果加到該位置
                marked_locations[location].append(result)

    # 在地圖上標註
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
              'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue']

    for i, (location, results) in enumerate(marked_locations.items()):
        coords = get_coordinates(location)
        if coords:
            lat, lon = coords

            # 建立 popup 內容
            popup_html = f"<div style='width: 300px;'>"
            popup_html += f"<h4 style='color: #2c3e50;'>{location}</h4>"
            popup_html += f"<p style='color: #7f8c8d; font-size: 12px;'>找到 {len(results)} 筆相關結果</p>"

            for j, res in enumerate(results, 1):
                popup_html += f"<div style='margin: 10px 0; padding: 10px; background: #ecf0f1; border-radius: 5px;'>"
                popup_html += f"<b style='color: #2980b9;'>[{j}] {res['title'][:50]}...</b><br>"
                popup_html += f"<small style='color: #34495e;'>{res['snippet'][:100]}...</small><br>"
                popup_html += f"<a href='{res['link']}' target='_blank' style='color: #3498db;'>查看連結</a>"
                popup_html += "</div>"

            popup_html += "</div>"

            # 加上標記
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_html, max_width=350),
                tooltip=f"{location} ({len(results)} 筆結果)",
                icon=folium.Icon(color=colors[i % len(colors)], icon='info-sign')
            ).add_to(m)

    return m


def display_results_summary(analyzed_results):
    """
    顯示分析結果摘要
    """
    print("\n" + "="*80)
    print(f"找到 {len(analyzed_results)} 筆包含地理位置的搜尋結果")
    print("="*80 + "\n")

    for i, result in enumerate(analyzed_results, 1):
        print(f"【結果 {i}】")
        print(f"標題: {result['title']}")
        print(f"地點: {', '.join(result['locations'])}")
        print(f"描述: {result['snippet'][:100]}...")
        print(f"網址: {result['link']}")
        print("-"*80 + "\n")

## 5.搜尋並標註地圖

In [5]:
def search_and_map(query, api_key, num_results=10):
    """
    整合搜尋與地圖標註的主函數

    參數:
        query (str): 搜尋關鍵字
        api_key (str): SerpAPI 金鑰
        num_results (int): 搜尋結果數量

    回傳:
        folium.Map: 標註好的地圖
    """
    print(f"正在搜尋: {query}\n")

    # 1. 執行搜尋
    results = search_google(query, api_key, num=num_results)

    if not results:
        print("沒有找到任何結果")
        return None

    print(f"✓ 找到 {len(results)} 筆搜尋結果")

    # 2. 分析結果並擷取地理位置
    analyzed_results = analyze_results_with_locations(results)

    if not analyzed_results:
        print("\n⚠️ 搜尋結果中沒有找到可辨識的台灣地區")
        return None

    print(f"✓ 從結果中找到 {len(analyzed_results)} 筆包含地理位置的資料\n")

    # 3. 顯示結果摘要
    display_results_summary(analyzed_results)

    # 4. 建立地圖
    print("正在建立地圖...\n")
    map_obj = create_map_from_results(analyzed_results)
    print("✓ 地圖建立完成！")

    return map_obj

## 6. 使用範例

In [6]:
#搜尋台灣美食
query = "台灣必吃美食推薦"
map_result = search_and_map(query, SERPAPI_KEY, num_results=10)

if map_result:
    display(map_result)

正在搜尋: 台灣必吃美食推薦

✓ 找到 9 筆搜尋結果
✓ 從結果中找到 2 筆包含地理位置的資料


找到 2 筆包含地理位置的搜尋結果

【結果 1】
標題: 【台灣美食】從南吃到北！全台必吃美食大集合
地點: 台北, 基隆
描述: 台灣美食 · 基隆美食不只廟口？！必吃排骨飯、魷魚羹、龍蝦麻糬別錯過 基隆是許多台北人週末出門的好地方，不管是想欣賞海景還是大啖美食，基隆通通都能滿足。 · 【基隆名產】在 ......
網址: https://www.funtime.com.tw/theme/taiwan-food
--------------------------------------------------------------------------------

【結果 2】
標題: 臺北飲食攻略
地點: 臺北
描述: 小籠包 ... 一層層巧手摺疊的麵衣，包裹著肉質鮮嫩的內餡，大口咬下，鮮甜的金黃湯汁在嘴裡流竄，這道源於江南的著名麵點，在臺灣發揚光大，是許多國際觀光客來臺指名必吃的美食。...
網址: https://www.travel.taipei/zh-tw/must-visit/tasterguide
--------------------------------------------------------------------------------

正在建立地圖...

✓ 地圖建立完成！


In [7]:
#搜尋台灣景點
query = "台灣熱門觀光景點"
map_result = search_and_map(query, SERPAPI_KEY, num_results=10)

if map_result:
    display(map_result)

正在搜尋: 台灣熱門觀光景點

✓ 找到 9 筆搜尋結果
✓ 從結果中找到 6 筆包含地理位置的資料


找到 6 筆包含地理位置的搜尋結果

【結果 1】
標題: 觀光景點
地點: 宜蘭, 日月潭
描述: 北起的東北角、北海岸及宜蘭地區，擁有各類特殊海岸地形，沿途山青水秀，天藍海深；往中南部地區，日月潭的湖光山色、八卦山賞鷹、水果之鄉的梨山、佛教聖地的獅頭山等，都可充分 ......
網址: https://www.taiwan.net.tw/m1.aspx?sNo=0001016
--------------------------------------------------------------------------------

【結果 2】
標題: 2025台灣景點一日遊懶人包，國內旅遊、自由行景點
地點: 桃園
描述: 2025推薦超過60個「桃園景點」推薦懶人包，從觀光工廠玩到森林瀑布一日遊，桃園市除了到拉拉山玩之外，千萬千萬不要忘了安排桃園景點一日遊，直接整理推薦超過 ......
網址: https://bunnyann.tw/attractions/
--------------------------------------------------------------------------------

【結果 3】
標題: 2025 年台灣絕美秘境：嚴選40 個從北到南的必去推薦景點
地點: 台北, 北投
描述: 推薦【台北必去秘境】圓山大飯店密道 · 推薦【台北內湖必去秘境】大溝溪生態治水園區 · 推薦【台北公館必去秘境】台電加羅林魚木 · 推薦【台北北投必去秘境】 ......
網址: https://hshsharehouse.com/taiwan-hidden-gems-40-must-visit-destinations-north-to-south/
--------------------------------------------------------------------------------

【結果 4】
標題: 台灣旅遊熱門景點一日/二日遊、國內跟團團體行程
地點: 日月潭, 阿里山
描述: 台灣一日遊、兩天一夜、三天兩夜跟團行程，全台熱門景點：武陵農場、日月潭、司馬庫斯、阿里山等，跟團玩免開車

In [8]:
#自訂搜尋
custom_query = input("請輸入搜尋關鍵字 (包含地點相關): ")
num_results = int(input("幾筆結果? (5-20): "))

map_result = search_and_map(custom_query, SERPAPI_KEY, num_results=num_results)

if map_result:
    display(map_result)

請輸入搜尋關鍵字 (包含地點相關): 台北
幾筆結果? (5-20): 3
正在搜尋: 台北

✓ 找到 3 筆搜尋結果
✓ 從結果中找到 3 筆包含地理位置的資料


找到 3 筆包含地理位置的搜尋結果

【結果 1】
標題: 臺北市- 維基百科，自由的百科全書
地點: 臺北, 新北, 信義區
描述: 臺北市，通稱臺北，是中華民國的直轄市及首都，位於臺灣北部，地處臺北盆地東北半部，外圍由新北市環繞 :1。全市轄有12區，其中市政府所在地為信義區。...
網址: https://zh.wikipedia.org/zh-tw/%E8%87%BA%E5%8C%97%E5%B8%82
--------------------------------------------------------------------------------

【結果 2】
標題: 臺北旅遊網- 台北
地點: 台北, 臺北
描述: 在臺北，您每個所到之處，多樣的文化特質都充沛鼓動著。雕龍畫棟的廟宇與現代的街道完美吻合，還有許多世界級餐廳隨時提供您最正統的各式中華料理。別忘了，美味的夜市小吃 ......
網址: https://www.travel.taipei/zh-tw
--------------------------------------------------------------------------------

【結果 3】
標題: 台北自由行旅遊攻略2025:景點推薦
地點: 台北, 臺北, 101, 台北101, 西門町, 北投
描述: 台北熱門景點 · 台北101 · 西門町 · 陽明山 · 北投區 · 臺北市兒童新樂園 · 台北車站 · 國立故宮博物院 · 臺北市立動物園. 歡迎來到台北市立 ......
網址: https://www.klook.com/zh-TW/destination/c19-taipei/
--------------------------------------------------------------------------------

正在建立地圖...

✓ 地圖建立完成！


## 7. 儲存地圖(供參考，詳細請照作業需求)


In [9]:
# 將地圖儲存為 HTML 檔案
if map_result:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"search_map_{timestamp}.html"
    map_result.save(filename)
    print(f"\n✓ 地圖已儲存為: {filename}")
    print("可以用瀏覽器開啟檢視！")


✓ 地圖已儲存為: search_map_20251116_072426.html
可以用瀏覽器開啟檢視！


In [10]:
import os
import re
import json
from datetime import datetime
import pandas as pd
from IPython.display import display # 僅為兼容 Colab/Jupyter

# 確保這些套件已安裝: pip install serpapi folium geopy gradio pandas -q
try:
    from serpapi import GoogleSearch
    import folium
    from geopy.geocoders import Nominatim
    import gradio as gr
except ImportError as e:
    print(f"錯誤：缺少必要的函式庫。請執行 `!pip install serpapi folium geopy gradio pandas -q`。錯誤訊息: {e}")
    # 在實際運行環境中，這裡可能需要退出或提示安裝
    exit()

# ==============================================================================
# 1. 配置與初始化
# ==============================================================================
# ⚠️ 請替換成您實際的 SerpAPI 金鑰！
SERPAPI_KEY = "fa1097a76bf103c7a52fb91c2d7c58129281c4ec8b63890e88cf355d30f21668"

# 初始化地理編碼器
geolocator = Nominatim(user_agent="search_map_app_with_gradio")
COORD_CACHE = {} # 經緯度快取

# 台灣地點列表 (根據您的原始程式碼完整還原)
TAIWAN_LOCATIONS = [
    # 直轄市
    '台北', '台北市', '臺北', '臺北市', 'Taipei', '新北', '新北市', 'New Taipei',
    '桃園', '桃園市', 'Taoyuan', '台中', '台中市', '臺中', '臺中市', 'Taichung',
    '台南', '台南市', '臺南', '臺南市', 'Tainan', '高雄', '高雄市', 'Kaohsiung',
    # 縣市
    '基隆', '基隆市', 'Keelung', '新竹', '新竹市', '新竹縣', 'Hsinchu',
    '苗栗', '苗栗縣', 'Miaoli', '彰化', '彰化縣', 'Changhua', '南投', '南投縣', 'Nantou',
    '雲林', '雲林縣', 'Yunlin', '嘉義', '嘉義市', '嘉義縣', 'Chiayi',
    '屏東', '屏東縣', 'Pingtung', '宜蘭', '宜蘭縣', 'Yilan', '花蓮', '花蓮縣', 'Hualien',
    '台東', '台東縣', '臺東', '臺東縣', 'Taitung', '澎湖', '澎湖縣', 'Penghu',
    '金門', '金門縣', 'Kinmen', '連江', '連江縣', '馬祖', 'Matsu',
    # 知名景點/地標
    '101', '台北101', '西門町', '信義區', '中正區', '士林', '北投', '淡水', '九份', '平溪',
    '日月潭', '阿里山', '墾丁', '太魯閣', '東大門', '逢甲', '一中街', '勤美',
]

# ==============================================================================
# 2. 核心邏輯函式
# ==============================================================================

def search_google(query, api_key, num=5, gl='tw', hl='zh-tw'):
    """使用 SerpAPI 進行 Google 搜尋"""
    try:
        params = {'q': query, 'api_key': api_key, 'num': num, 'gl': gl, 'hl': hl}
        search = GoogleSearch(params)
        results = search.get_dict()
        return results.get('organic_results', [])
    except Exception as e:
        print(f"搜尋時發生錯誤: {e}")
        return []

def extract_locations_from_text(text):
    """從文字中擷取地區名稱"""
    found_locations = []
    text_lower = text.lower()
    for location in TAIWAN_LOCATIONS:
        if location.lower() in text_lower:
            base_name = location.replace('市', '').replace('縣', '').replace('臺', '台')
            if base_name not in [loc.replace('市', '').replace('縣', '').replace('臺', '台') for loc in found_locations]:
                found_locations.append(location)
    return found_locations

def get_coordinates(location_name):
    """取得地點的經緯度座標 (使用快取)"""
    if location_name in COORD_CACHE: return COORD_CACHE[location_name]
    try:
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            coords = (location.latitude, location.longitude)
            COORD_CACHE[location_name] = coords
            return coords
        return None
    except Exception as e:
        print(f"無法取得 {location_name} 的座標: {e}")
        return None

def analyze_results_with_locations(results):
    """分析搜尋結果並擷取地理位置資訊"""
    analyzed_results = []
    for result in results:
        title = result.get('title', '')
        snippet = result.get('snippet', '')
        combined_text = f"{title} {snippet}"
        locations = extract_locations_from_text(combined_text)
        if locations:
            analyzed_results.append({
                'title': title, 'snippet': snippet, 'link': result.get('link', ''), 'locations': locations
            })
    return analyzed_results

def create_map_from_results(analyzed_results, center=[23.5, 121.0], zoom=7):
    """根據分析結果建立 Folium 地圖，並將其儲存為 HTML 檔案路徑"""
    m = folium.Map(location=center, zoom_start=zoom, tiles='OpenStreetMap')
    marked_locations = {}

    for result in analyzed_results:
        for location in result['locations']:
            coords = get_coordinates(location)
            if coords:
                if location not in marked_locations: marked_locations[location] = []
                marked_locations[location].append(result)

    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
              'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue']

    for i, (location, results) in enumerate(marked_locations.items()):
        coords = get_coordinates(location)
        if coords:
            lat, lon = coords
            color_index = i % len(colors)

            # 建立 Popup 內容
            popup_html = f"<div style='width: 300px; font-family: Arial, sans-serif;'>"
            popup_html += f"<h4 style='color: #2c3e50; margin-bottom: 5px;'>{location}</h4>"
            popup_html += f"<p style='color: #7f8c8d; font-size: 12px;'>找到 {len(results)} 筆相關結果</p><hr style='margin: 5px 0;'>"
            for j, res in enumerate(results, 1):
                popup_html += f"<div style='margin-bottom: 8px; border-left: 3px solid {colors[color_index]}; padding-left: 5px;'>"
                popup_html += f"<b style='color: #2980b9;'>[{j}] {res['title'][:50]}...</b><br>"
                popup_html += f"<small style='color: #34495e;'>{res['snippet'][:100]}...</small><br>"
                popup_html += f"<a href='{res['link']}' target='_blank' style='color: #3498db; font-size: 11px;'>查看連結</a>"
                popup_html += "</div>"
            popup_html += "</div>"

            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_html, max_width=350),
                tooltip=f"{location} ({len(results)} 筆結果)",
                icon=folium.Icon(color=colors[color_index], icon='info-sign')
            ).add_to(m)

    map_filename = f"map_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
    m.save(map_filename)
    return map_filename

def export_results_to_csv(analyzed_results_list):
    """將分析後的結果列表轉換為 CSV 檔案 (模擬 Sheets I/O)"""
    if not analyzed_results_list:
        return None

    export_data = []
    for item in analyzed_results_list:
        export_data.append({
            'Title': item['title'],
            'Snippet': item['snippet'],
            'Link': item['link'],
            'Locations_Found': ', '.join(item['locations'])
        })

    df = pd.DataFrame(export_data)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_filename = f"search_results_{timestamp}.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    return csv_filename

# ==============================================================================
# 3. Gradio 專用主函數
# ==============================================================================

def search_and_map_gradio(query, num_results):
    """
    Gradio 專用主函數，返回摘要文本、地圖路徑和 CSV 路徑。
    """
    if not query:
        return "⚠️ 請輸入搜尋關鍵字。", None, None

    # 1. 執行搜尋與分析
    results = search_google(query, SERPAPI_KEY, num=num_results)
    analyzed_results = analyze_results_with_locations(results)

    # 2. 準備顯示結果 (Markdown 格式)
    summary_text = f"## 🔍 搜尋結果分析報告\n"
    summary_text += f"**關鍵字:** `{query}`\n"
    summary_text += f"**找到總結果:** {len(results)} 筆\n"
    summary_text += f"**包含地點的結果:** **{len(analyzed_results)}** 筆\n"
    summary_text += "\n---\n"

    if not analyzed_results:
        summary_text += "\n⚠️ 搜尋結果中沒有找到可辨識的台灣地區，無法在地圖上標註。"
        return summary_text, None, None

    # 準備詳細結果的表格 (Markdown Table)
    table_data = "| 序號 | 標題 (點擊查看連結) | 地點 |\n| :---: | :--- | :--- |\n"
    for i, result in enumerate(analyzed_results, 1):
        link_title = f"[{result['title'][:40]}... ]({result['link']})"
        locations_str = ', '.join(result['locations'])
        table_data += f"| {i} | {link_title} | **{locations_str}** |\n"
    summary_text += table_data

    # 3. 建立地圖
    map_path = create_map_from_results(analyzed_results)

    # 4. 準備 CSV 檔案 (模擬 Sheets I/O)
    csv_path = export_results_to_csv(analyzed_results)

    # 返回摘要、地圖路徑和 CSV 路徑
    return summary_text, map_path, csv_path

# ==============================================================================
# 4. Gradio 介面定義與啟動
# ==============================================================================

with gr.Blocks(title="台灣地點搜尋與地圖視覺化工具") as demo:
    gr.Markdown("# 🗺️ 台灣地點搜尋與地圖視覺化工具 (專題實作)")
    gr.Markdown("此應用整合了網路搜尋、地理編碼、地圖視覺化，並提供資料匯出功能。")
    gr.Markdown("* **功能狀態：** **✅** 地圖入Gradio | **✅** 程式碼修改 | **🔺** I/O to GSheets (模擬為 CSV 下載)。")

    with gr.Row():
        query_input = gr.Textbox(label="🔍 搜尋關鍵字", value="台灣熱門觀光景點", lines=1)
        num_input = gr.Slider(minimum=5, maximum=20, step=1, value=10, label="結果數量", interactive=True)
        search_button = gr.Button("🚀 開始搜尋與分析", variant="primary")

    with gr.Row():
        with gr.Column(scale=1):
            summary_output = gr.Markdown("📝 **摘要與結果列表**：點擊上方按鈕開始搜尋。")

            # CSV/Sheets I/O 輸出元件
            csv_file_link = gr.File(label="📥 下載分析結果 (CSV 格式，方便匯入 Google Sheets/Excel)", visible=True)
            map_file_link = gr.File(label="📄 下載互動式地圖 HTML 檔案", visible=True)

        with gr.Column(scale=2):
            map_display = gr.HTML(label="🌍 地圖視覺化")

    # 邏輯串接 (點擊 -> 搜尋 -> 輸出三個結果)
    search_button.click(
        fn=search_and_map_gradio,
        inputs=[query_input, num_input],
        outputs=[summary_output, map_file_link, csv_file_link]
    ).then(
        # 將地圖檔案路徑轉換為 HTML iframe 進行渲染
        fn=lambda file_path: f'<iframe src="file={file_path}" width="100%" height="500px"></iframe>' if file_path else "無法生成地圖或未找到地點。",
        inputs=[map_file_link],
        outputs=[map_display]
    )

if __name__ == "__main__":
    print("\n-------------------------------------------")
    print("Gradio 應用程式啟動中...")
    print("-------------------------------------------\n")

    # 使用 share=True 可以在 Colab 或遠端伺服器上生成公共連結
    demo.launch(share=True)


-------------------------------------------
Gradio 應用程式啟動中...
-------------------------------------------

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ded42fef8fa34b3469.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
